# Querying Files

Here it is the schema for the bookstore dataset used in this notebook:

![bookstore dataset schema](../Includes/images/image1.png)

There are three tables:
* customers
* orders
* books

The customer data is on JSON format. Downloading and copying the datasets to Databricks:

In [0]:
%run ../Includes/Copy-Datasets

## Reading JSON files

The customer data is on JSON format. Listing the file in the customers directory:

In [0]:
%python
files = dbutils.fs.ls(f"{dataset_bookstore}/customers-json")
display(files)

There are 6 JSON files in this directory. Let's read them!

Querying a JSOn file:

In [0]:
SELECT * FROM json.`dbfs:/mnt/demo-datasets/bookstore/customers-json/export_001.json`

This table has different columns:
* customer_id
* email
* profile (a JSON string)
* updated (timestamp)

There are 300 records in the the source file.

To query simultaneously all JSON files * can be used:

In [0]:
SELECT * FROM json.`dbfs:/mnt/demo-datasets/bookstore/customers-json/export_*.json`

Additionally, all the files contained in a directory can be queried assuming all the files have the same schema and format:

In [0]:
SELECT * FROM json.`dbfs:/mnt/demo-datasets/bookstore/customers-json/`

How many customers are there?

In [0]:
SELECT count(*) FROM  json.`dbfs:/mnt/demo-datasets/bookstore/customers-json/`

There are 1700 customers in this dataset.

When reading multiple files, it is useful to add the `input_file_name` function, whcih is built-in Spark SQL command that **records the source data file for each record**. It can be especially helpful if troublesahooting problems in the source data become necessary.

In [0]:
SELECT *,
  input_file_name() source_file
  FROM json.`dbfs:/mnt/demo-datasets/bookstore/customers-json/`

In addition to the columns, the source file column has been added, which is really helpful.

Another interesting option is to use the `text` format, which allows **to query any text-based files** (JSON, CSV, TSV, TXT, ...).

In [0]:
SELECT * FROM text.`dbfs:/mnt/demo-datasets/bookstore/customers-json/`

Each of the file has been loaded as a row with one string column (value). This can be used when data would be corrupted and some custom text parsing function is needed to extract data.

Additionally, `binaryFile` can be used to extract the raw bytes and some metadata of files.

In [0]:
SELECT * FROM binaryFile.`dbfs:/mnt/demo-datasets/bookstore/customers-json/`

Some information has been given:
* path
* modificationTime
* length
* content (binary representation of the file)

## Reading CSV files

The books data is in CSV format.

In [0]:
SELECT * FROM csv.`dbfs:/mnt/demo-datasets/bookstore/books-csv`

The header row has been extracted as a table row, and all columns has been loaded in a single column. The delimeter (semicolon) of the file has to be taken into account. Fos files without a schema defined, there is another way to load files:

In [0]:
CREATE TABLE books_csv
  (book_id STRING, title STRING, author STRING, category STRING, price DOUBLE)
USING CSV
OPTIONS (
  header = "true",
  delimiter = ";"
)
LOCATION "dbfs:/mnt/demo-datasets/bookstore/books-csv"

In [0]:
SELECT * FROM books_csv

When working with CSV files as a data source, it is important to ensure that column oder does not change in case additional data files are added to the source directory. Spark will always load data and apply column names and data types in the order specified during table creation.

In [0]:
DESCRIBE EXTENDED books_csv

The table provider is CSV, that means no data has moved during table creation. Files stored in an external location are being pointed. Additionally, all metadata and options passed during table creation will persist ensuring data will always be read with these options.

Adding CSV files to the directory:

In [0]:
%python
files = dbutils.fs.ls(f"{dataset_bookstore}/books-csv")
display(files)

There are 4 CSV files in the directory.

**Spark Dataframe API** allows to write data in a specific format (like .csv).

In [0]:
%python

(spark.read
 .table("books_csv")
 .write
 .mode("append")
 .format("csv")
 .option('header', 'true')
 .option('delimiter', ';')
 .save(f"{dataset_bookstore}/books-csv")
)

In [0]:
%python
files = dbutils.fs.ls(f"{dataset_bookstore}/books-csv")
display(files)

There are extra .csv files that have been written to the directory by Spark.

In [0]:
SELECT COUNT(*) FROM books_csv

Even with the new data succesfully written to the table directory, the new data is still hidden, as there should be 24 files. Spark automatically cashed the underlying data in local storage to ensure that on subsequent queries, Spark will provide the optimal performance by just querying this local cache.

This external .csv file is not configured to tell Spark that it should refresh this data.

In [0]:
REFRESH TABLE books_csv

Refreshing tha table will invalidate its cache. That means it will be needed to rescan the original data source and pull data back into memory. For a very large dataset, this might take a significant amount of time.

In [0]:
SELECT COUNT(*) FROM books_csv

Now the number of books is 24. Therefore, non-Delta Tables have some limitations. To create Delta Tables to load data from external sources, **CTAS statements** are used.

In [0]:
CREATE TABLE customers AS
SELECT * FROM json.`dbfs:/mnt/demo-datasets/bookstore/customers-json/`;

DESCRIBE EXTENDED customers

This extracts the data from the JSON file and load them into the table `customers`. In the metadata is shown that a Delta Table has been created and it is also a MANAGED table. Additionally, the schema has been inferred automatically from the query results, because **CTAS stataments automatically infer schema information from query results and do not support manual schema declarations**.

**CTAS statements** are useful for **external data ingestion from sources with well-defined schema** such as parquet files and tables.

In addition, CTAS statements do NOT support specifying additional file options, which adds significant limitations when trying to ingest data from .csv files.

In [0]:
CREATE TABLE books_unparsed AS
SELECT * FROM csv.`dbfs:/mnt/demo-datasets/bookstore/books-csv`;

SELECT * FROM books_unparsed

A Delta Table has been created, however, the data is not well parsed. To correct that, a reference is needed to specify options.

In [0]:
CREATE TEMP VIEW books_tmp_view
  (book_id STRING, title STRING, author STRING, category STRING, price DOUBLE)
USING CSV
OPTIONS (
  path = "dbfs:/mnt/demo-datasets/bookstore/books-csv/export_*.csv",
  header = "true",
  delimiter = ";"
);

CREATE TABLE books AS
  SELECT * FROM books_tmp_view;

SELECT * FROM books

The table has been succesfully created, and only 12 records have been retrieved, due to the use of the wildcard character (*) in the path location.

In [0]:
DESCRIBE EXTENDED books